In [6]:
import yfinance as yf

# 定義股票代號
ticker = "2882.TW"

# 獲取股票資訊
stock = yf.Ticker(ticker)
hist = stock.history(period="6mo")

# 獲取最近一年的財報數據
financials = stock.financials
balance_sheet = stock.balance_sheet
cashflow = stock.cashflow

# 獲取股息收益率
dividends = stock.dividends
dividend_yield = dividends[-1] / stock.history(period="1d")['Close'][-1] * 100

# 獲取每股盈餘 (EPS)
eps = stock.info.get('trailingEps', 'N/A')

# 獲取市盈率 (P/E Ratio)
pe_ratio = stock.info.get('trailingPE', 'N/A')


# 構建財務數據字典
financial_data = {
    "revenue": financials.loc['Total Revenue'][-1] / 1e9,  # 轉換為億
    "net_income": financials.loc['Net Income'][-1] / 1e9,  # 轉換為億
    "pe_ratio": pe_ratio,
    "eps": eps,
    "stock_price": stock.history(period="1d")['Close'][-1],
    "dividend_yield": dividend_yield,
}

print(financial_data)

{'revenue': 637.379349, 'net_income': 74.57946, 'pe_ratio': 10.8867, 'eps': 6.09, 'stock_price': 66.30000305175781, 'dividend_yield': 3.0165911130331}


/var/folders/0l/dmc60s_975jff_p_2_n34cq40000gq/T/ipykernel_92480/3440644423.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dividend_yield = dividends[-1] / stock.history(period="1d")['Close'][-1] * 100
/var/folders/0l/dmc60s_975jff_p_2_n34cq40000gq/T/ipykernel_92480/3440644423.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dividend_yield = dividends[-1] / stock.history(period="1d")['Close'][-1] * 100
/var/folders/0l/dmc60s_975jff_p_2_n34cq40000gq/T/ipykernel_92480/3440644423.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be 

In [7]:
import openai
import dotenv
import os

dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_stock_analysis(ticker, financial_data):
    """
    使用 OpenAI API 生成股票分析報告
    :param ticker: 股票代碼
    :param financial_data: 股票的關鍵財務數據（字典格式）
    :return: 生成的分析報告
    """
    prompt = f"""
    我想要一份股票分析報告。以下是 {ticker} 的財務數據：
    收入: {financial_data.get('revenue', 'N/A')} 億美元
    淨利潤: {financial_data.get('net_income', 'N/A')} 億美元
    市盈率: {financial_data.get('pe_ratio', 'N/A')}
    股價: {financial_data.get('stock_price', 'N/A')} 美元
    
    你是一位專業的投資經理人，請根據以上數據撰寫一份專業的股票分析報告，包含對該公司財務健康狀況的評論、未來增長的可能性，以及投資建議。
    """

    parameters = {
        "engine": "gpt-3.5-turbo-instruct",  # 選擇適當的模型
        "max_tokens": 500,
        "temperature": 0.7,
        "prompt": prompt
    }

    print(parameters)
    
    try:
        response = openai.Completion.create(
            **parameters
        )
        return response['choices'][0]['text'].strip()
    except Exception as e:
        return f"生成失敗: {e}"

# 測試範例數據
sample_data = financial_data

analysis_report = generate_stock_analysis(ticker, sample_data)


{'engine': 'gpt-3.5-turbo-instruct', 'max_tokens': 500, 'temperature': 0.7, 'prompt': '\n    我想要一份股票分析報告。以下是 2882.TW 的財務數據：\n    收入: 637.379349 億美元\n    淨利潤: 74.57946 億美元\n    市盈率: 10.8867\n    股價: 66.30000305175781 美元\n    \n    你是一位專業的投資經理人，請根據以上數據撰寫一份專業的股票分析報告，包含對該公司財務健康狀況的評論、未來增長的可能性，以及投資建議。\n    '}


In [8]:
def export_to_markdown(ticker, analysis_report, output_file="stock_analysis.md"):
    """
    將分析報告匯出為 Markdown 文件
    """
    with open(output_file, "w", encoding="utf-8") as file:
        # HackMD 格式化內容
        file.write(f"# {ticker} 股票分析報告\n\n")
        file.write("## 摘要\n")
        file.write("以下是基於公司財務數據的專業分析報告。\n\n")
        file.write("## 詳細報告\n")
        file.write(analysis_report)
    print(f"報告已成功匯出到 {output_file}")


export_to_markdown(ticker,analysis_report, "week12/stock_analysis.md")

報告已成功匯出到 week12/stock_analysis.md
